In [11]:
import yaml
import numpy as np
import pandas as pd

import cv2
import albumentations as A

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold

from utils import obj

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

pd.options.display.max_columns=100

In [3]:
data_path = '../open/'
info = pd.read_excel(data_path+'clinical_info.xlsx')

df_train = pd.read_csv(data_path+'train.csv')
df_test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

df_train['img_path'] = df_train['img_path'].apply(lambda x: x.replace('./', '../open/'))
df_test['img_path'] = df_test['img_path'].apply(lambda x: x.replace('./', '../open/'))

In [ ]:
for i, p in enumerate(df_train['img_path']):
    img = cv2.imread(p)
    trans = A.Resize(224, 224)
    img = trans(image=img)['image']
    if size_ls[i][0]>3500:
        

        plt.imshow(img)
        plt.show()

In [41]:
size_ls = []

for i in df_train['img_path']:
    img = cv2.imread(i)
    # trans = A.Compose([
    #                     A.Resize(224, 224), 
                    #    A.HorizontalFlip(p=1),
                    #    A.CoarseDropout(p=1),
                    #    A.Cutout(max_h_size=int(224*0.1), max_w_size=int(224*0.1), num_holes=5, p=0.8),
                    #    ])
    # img = trans(image=img)['image']
    # plt.imshow(img)
    # plt.show()
    size_ls += [img.shape]

In [3]:
info

,항목,코딩,설명
0,ID,NaN,부여번호
1,나이,NaN,진단시 나이
2,수술연월일,연-월-일,NaN
3,진단명,1: ductal\n2: lobular\n3. mucinous \n4. other,histologic type
4,암의 위치,1: Right\n2: Left\n3: Both,NaN
5,암의 개수,1: single\n2: multiple,NaN
6,암의 장경,NaN,"암 Size (mm), 여러 개의 암일 경우 가장 큰 종양의 장경"
7,NG,1: NG1\n2: NG2\n3: NG3,"1은 핵의 모양이 정상에 가까운 경우, 2는 중간, 3은 정상으로부터 매우 달라진경..."
8,HG,1: HG1\n2: HG2\n3: HG3\n4. not graded,"Grade 1 (scores of 3, 4, or 5): 조직학적 분화도가 좋아서 ..."
9,HG_score_1,1: score 1\n2: score 2\n3: score 3\n4: not graded,(Tubule formation)\nScore 1 (>75% of tumor are...


In [4]:
df_train = pd.read_csv(data_path+'train.csv')
df_test = pd.read_csv(data_path+'test.csv')

# preprocess outlier
df_train['PR_Allred_score'] = df_train['PR_Allred_score'].where((0<=df_train['PR_Allred_score']) & (df_train['PR_Allred_score']<=8))

for col in ['NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_type', 'ER_Allred_score', 'PR_Allred_score', 'HER2_SISH_ratio']:
    df_train[col].fillna(0, inplace=True)
    df_test[col].fillna(0, inplace=True)

df_test['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)
df_train['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)

df_train['BRCA_mutation'] = df_train['BRCA_mutation'].fillna(1)
df_test['BRCA_mutation'] = df_test['BRCA_mutation'].fillna(1)

for col in ['T_category', 'HER2', 'HER2_IHC', 'HER2_SISH', 'KI-67_LI_percent']:
    df_train[col].fillna(-1, inplace=True)
    df_test[col].fillna(-1, inplace=True)

    df_train[col]+=1
    df_test[col]+=1

df_train[['ER', 'PR']] = df_train[['ER', 'PR']].fillna(0)
# remove ER, PR nan value
# df_train = df_train.drop(266).reset_index(drop=True)

In [81]:
df_train = pd.read_csv(data_path+'df_train.csv')
df_test = pd.read_csv(data_path+'df_test.csv')

# preprocess outlier
# df_train['PR_Allred_score'] = df_train['PR_Allred_score'].where((0<=df_train['PR_Allred_score']) & (df_train['PR_Allred_score']<=8))

# for col in ['NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_type', 'ER_Allred_score', 'PR_Allred_score', 'HER2_SISH_ratio']:
#     df_train[col].fillna(0, inplace=True)
#     df_test[col].fillna(0, inplace=True)

# df_test['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)
# df_train['암의 장경'].fillna(df_train['암의 장경'].median(), inplace=True)

# df_train['BRCA_mutation'] = df_train['BRCA_mutation'].fillna(1)
# df_test['BRCA_mutation'] = df_test['BRCA_mutation'].fillna(1)

# for col in ['T_category', 'HER2', 'HER2_IHC', 'HER2_SISH', 'KI-67_LI_percent']:
#     df_train[col].fillna(-1, inplace=True)
#     df_test[col].fillna(-1, inplace=True)

#     df_train[col]+=1
#     df_test[col]+=1

# # remove ER, PR nan value
# df_train = df_train.drop(266).reset_index(drop=True)

FOLDS = 5

def basic_set():
    use_features = [
                    '나이', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                    'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 
                    'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 
                    'KI-67_LI_percent', 
                    'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 
                    'img_height', 'img_width',
                    # 'BRCA_mutation2',
                    ]

    cat_features = ['나이', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1',
                            'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type',
                            'T_category', 'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score',
                            'HER2', 'HER2_IHC', 'HER2_SISH','BRCA_mutation']
    cat_features = ['진단명']
    return use_features, cat_features

df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year
df_test['due_date'] = 2022 - pd.to_datetime(df_test['수술연월일']).dt.year
# df_train['due_date2'] = (2022-pd.to_datetime(df_train['수술연월일']).dt.year)*12 + pd.to_datetime(df_train['수술연월일']).dt.month
# df_test['due_date2'] = (2022-pd.to_datetime(df_test['수술연월일']).dt.year)*12 + pd.to_datetime(df_test['수술연월일']).dt.month

df_train['date_year'] = pd.to_datetime(df_train['수술연월일']).dt.year
df_test['date_year'] = pd.to_datetime(df_test['수술연월일']).dt.year

# df_train['암의 장경/개수'] = df_train['암의 장경'] / df_train['암의 개수']
# df_test['암의 장경/개수'] = df_test['암의 장경'] / df_test['암의 개수']

skf = StratifiedKFold(n_splits=FOLDS, random_state=42, shuffle=True)
splits = list(skf.split(df_train, df_train['N_category']))
# labels = df_train['N_category'].astype(str) + '_' + df_train['due_date'].astype(str)
# splits = list(skf.split(df_train, labels))


fi_df = pd.DataFrame()
oof = np.zeros(len(df_train))
preds = np.zeros(len(df_test))

for fold in range(FOLDS):
    print('start fold :', fold)
    test = df_test.copy()
    
    use_features, cat_features = basic_set()
    use_features += ['due_date', 'date_year']
    # use_features += ['due_date', 'date_year', '암의 장경/개수']

    tt = df_train.loc[splits[fold][0], use_features].reset_index(drop=True)
    tt_target = df_train.loc[splits[fold][0], 'N_category'].values
    vv = df_train.loc[splits[fold][1], use_features].reset_index(drop=True)
    vv_target = df_train.loc[splits[fold][1], 'N_category'].values

    for df in [tt, vv, test]:
        df['나이'] = pd.cut(df['나이'], [20, 30, 40, 50, 60, 70, 80, 100])
        # df['암의 장경'] = pd.cut(df['암의 장경'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
        df['암의 장경'] = pd.cut(df['암의 장경'], [0, 20, 40, 60, 80, 100, 200])
        # df['암의 장경/개수'] = pd.cut(df['암의 장경/개수'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
        # df['KI-67_LI_percent'] = pd.cut(df['KI-67_LI_percent'], [0, 20, 40, 60, 80, 100])
        
    # for col in ['나이', '암의 장경', '암의 장경/개수']:
    # for col in ['나이', '암의 장경', 'KI-67_LI_percent']:
    for col in ['나이', '암의 장경']:
        tmp_dict = {j:i for i,j in enumerate(np.unique(tt[col].astype(str)))}
        tt[col] = tt[col].astype(str).map(tmp_dict)
        vv[col] = vv[col].astype(str).map(tmp_dict)
        test[col] = test[col].astype(str).map(tmp_dict)
    
    # tt['암의 장경2'] = tt['암의 장경'].isnull()
    # vv['암의 장경2'] = vv['암의 장경'].isnull()
    # test['암의 장경2'] = test['암의 장경'].isnull()

    # for df in [tt, vv, test]:
    #     # df['나이_due_date'] = df['나이'].astype(str) + '_' + df['due_date'].astype(str)
    #     df['암'] = df['암의 위치'].astype(str) + '_' + df['암의 장경'].astype(str)
    # df = pd.concat([tt, vv])
    # tmp_dict = {v:i for i, v in enumerate(np.unique(df['암']))}
    # for col in ['암']:
    #     tt[col] = tt[col].astype(str).map(tmp_dict)
    #     vv[col] = vv[col].astype(str).map(tmp_dict)
    #     test[col] = test[col].astype(str).map(tmp_dict)
    
    # use_features += ['암']
    
    # tmp_dict = df_train.loc[splits[fold][0]].groupby('due_date')['N_category'].mean()
    # tt['진단명_target'] = tt['due_date'].map(tmp_dict)
    # vv['진단명_target'] = vv['due_date'].map(tmp_dict)
    # tmp_dict = df_train.groupby('due_date')['N_category'].mean()
    # test['진단명_target'] = test['due_date'].map(tmp_dict)
    # use_features += ['진단명_target']

    test = test[use_features].reset_index(drop=True)

    # lgb = LGBMClassifier(
    #                     n_estimators=5000,
    #                     learning_rate=0.03,
    #                     max_depth=-1,
    #                     num_leaves=64,
    #                     )

    # lgb.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=100)
    # oof[splits[fold][1]] += lgb.predict_proba(vv)[:, 1]
    # preds += lgb.predict_proba(test)[:, 1] / 5
    # fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, lgb.feature_importances_), columns=['feature', 'value'])])
    # tt[cat_features] = tt[cat_features].fillna(-99).astype(int)
    # vv[cat_features] = vv[cat_features].fillna(-99).astype(int)
    cat = CatBoostClassifier(
                            n_estimators=5000,
                            learning_rate=0.03,
                            # objective='logloss',
                            # eval_metric='F1',
                            # reg_lambda=0.01,
                            random_seed=1028,
                            allow_writing_files=False,
                            logging_level='Silent',
                            )
    cat.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=500,
            # cat_features=cat_features
            
            )
    oof[splits[fold][1]] += cat.predict_proba(vv)[:, 1]
    preds += cat.predict_proba(test)[:, 1] / FOLDS
    fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, cat.feature_importances_), columns=['feature', 'value'])])

start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


In [82]:
f1_score(df_train['N_category'],
        np.where(oof>0.5, 1, 0),
        average='macro'
        )

0.8249704200009802

In [83]:
f1_score(df_train['N_category'],
        np.where(oof>0.5, 1, 0),
        average='macro'
        )

0.8249704200009802

In [84]:
a = pd.read_csv('../submit/best.csv')

In [85]:
sum(a['N_category']!=preds.round())

8

In [414]:
fi_df.groupby('feature').mean().reset_index().sort_values('value', ascending=False)

,feature,value
19,due_date,16.677493
18,date_year,13.121424
14,NG,11.432816
11,HG_score_2,6.966082
13,KI-67_LI_percent,5.587786
22,암의 위치,3.982920
16,PR_Allred_score,3.761313
23,암의 장경,3.517789
1,DCIS_or_LCIS_type,3.505500
20,나이,3.345240


In [428]:
pd.Series(np.where(oof>0.55, 1, 0)).value_counts(True)

0    0.533534
1    0.466466
dtype: float64

In [431]:
df_train['N_category'].value_counts(True)

1    0.514515
0    0.485485
Name: N_category, dtype: float64

In [433]:
pd.Series(np.where(preds>0.4, 1, 0)).value_counts(True)

1    0.512
0    0.488
dtype: float64

In [435]:
pd.Series(np.where(preds>0.4, 1, 0)).value_counts(True)

1    0.512
0    0.488
dtype: float64

In [79]:
sub['N_category'] = np.where(preds>=0.5, 1, 0)

In [80]:
sub.to_csv('../submit/catboost_img_wh.csv', index=False)

In [21]:
np.save('../submit/cat_oof.npy', oof)

In [22]:
np.save('../submit/cat_preds.npy', preds)

In [4]:
df_train = pd.read_csv(data_path+'df_train.csv')
df_test = pd.read_csv(data_path+'df_test.csv')

In [6]:
df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year
df_test['due_date'] = 2022 - pd.to_datetime(df_test['수술연월일']).dt.year

In [25]:
df_train['암의 위치']df_train['암의 개수'].astype(str)

0      2_1
1      1_1
2      2_1
3      2_1
4      2_1
      ... 
995    2_1
996    1_1
997    2_1
998    1_1
999    2_1
Length: 1000, dtype: object

In [15]:
seed_oof = []
seed_pred = []
for seed in [1028, 42, 204, 1510, 99]:

    df_train = pd.read_csv(data_path+'df_train.csv')
    df_test = pd.read_csv(data_path+'df_test.csv')

    FOLDS = 5

    def basic_set():
        use_features = [
                        '나이', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
                        'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 
                        'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score', 
                        'KI-67_LI_percent', 
                        'HER2', 'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation', 
                        # 'img_height', 'img_width',
                        ]

        cat_features = ['나이', '진단명', '암의 위치', '암의 개수', '암의 장경', 'NG', 'HG', 'HG_score_1',
                                'HG_score_2', 'HG_score_3', 'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type',
                                'T_category', 'ER', 'ER_Allred_score', 'PR', 'PR_Allred_score',
                                'HER2', 'HER2_IHC', 'HER2_SISH','BRCA_mutation']
        cat_features = ['진단명']
        return use_features, cat_features

    df_train['due_date'] = 2022 - pd.to_datetime(df_train['수술연월일']).dt.year
    df_test['due_date'] = 2022 - pd.to_datetime(df_test['수술연월일']).dt.year
    # df_train['due_date2'] = (2022-pd.to_datetime(df_train['수술연월일']).dt.year)*12 + pd.to_datetime(df_train['수술연월일']).dt.month
    # df_test['due_date2'] = (2022-pd.to_datetime(df_test['수술연월일']).dt.year)*12 + pd.to_datetime(df_test['수술연월일']).dt.month

    df_train['date_year'] = pd.to_datetime(df_train['수술연월일']).dt.year
    df_test['date_year'] = pd.to_datetime(df_test['수술연월일']).dt.year

    # df_train['암의 장경/개수'] = df_train['암의 장경'] / df_train['암의 개수']
    # df_test['암의 장경/개수'] = df_test['암의 장경'] / df_test['암의 개수']
    
    
    df_train['암의 위치__암의 개수'] = df_train['암의 위치'].map(df_train.groupby(['암의 위치'])['암의 개수'].mean())
    df_test['암의 위치__암의 개수'] = df_test['암의 위치'].map(df_train.groupby(['암의 위치'])['암의 개수'].mean())

    skf = StratifiedKFold(n_splits=FOLDS, random_state=seed, shuffle=True)
    # splits = list(skf.split(df_train, df_train['N_category']))
    labels = df_train['N_category'].astype(str) + '_' + df_train['due_date'].astype(str)
    splits = list(skf.split(df_train, labels))


    fi_df = pd.DataFrame()
    oof = np.zeros(len(df_train))
    preds = np.zeros(len(df_test))

    for fold in range(FOLDS):
        print('start fold :', fold)
        test = df_test.copy()
        
        use_features, cat_features = basic_set()
        use_features += ['due_date', 'date_year', '암의 위치__암의 개수']
        # use_features += ['due_date', 'date_year', '암의 장경/개수']

        tt = df_train.loc[splits[fold][0], use_features].reset_index(drop=True)
        tt_target = df_train.loc[splits[fold][0], 'N_category'].values
        vv = df_train.loc[splits[fold][1], use_features].reset_index(drop=True)
        vv_target = df_train.loc[splits[fold][1], 'N_category'].values

        for df in [tt, vv, test]:
            df['나이'] = pd.cut(df['나이'], [20, 30, 40, 50, 60, 70, 80, 100])
            # df['암의 장경'] = pd.cut(df['암의 장경'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
            df['암의 장경'] = pd.cut(df['암의 장경'], [0, 20, 40, 60, 80, 100, 200])
            # df['암의 장경/개수'] = pd.cut(df['암의 장경/개수'], [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200])
            # df['KI-67_LI_percent'] = pd.cut(df['KI-67_LI_percent'], [0, 20, 40, 60, 80, 100])
            
        # for col in ['나이', '암의 장경', '암의 장경/개수']:
        # for col in ['나이', '암의 장경', 'KI-67_LI_percent']:
        for col in ['나이', '암의 장경']:
            tmp_dict = {j:i for i,j in enumerate(np.unique(tt[col].astype(str)))}
            tt[col] = tt[col].astype(str).map(tmp_dict)
            vv[col] = vv[col].astype(str).map(tmp_dict)
            test[col] = test[col].astype(str).map(tmp_dict)
        
        # tt['암의 장경2'] = tt['암의 장경'].isnull()
        # vv['암의 장경2'] = vv['암의 장경'].isnull()
        # test['암의 장경2'] = test['암의 장경'].isnull()

        # for df in [tt, vv, test]:
        #     # df['나이_due_date'] = df['나이'].astype(str) + '_' + df['due_date'].astype(str)
        #     df['암'] = df['암의 위치'].astype(str) + '_' + df['암의 장경'].astype(str)
        # df = pd.concat([tt, vv])
        # tmp_dict = {v:i for i, v in enumerate(np.unique(df['암']))}
        # for col in ['암']:
        #     tt[col] = tt[col].astype(str).map(tmp_dict)
        #     vv[col] = vv[col].astype(str).map(tmp_dict)
        #     test[col] = test[col].astype(str).map(tmp_dict)
        
        # use_features += ['암']
        
        # tmp_dict = df_train.loc[splits[fold][0]].groupby('due_date')['N_category'].mean()
        # tt['진단명_target'] = tt['due_date'].map(tmp_dict)
        # vv['진단명_target'] = vv['due_date'].map(tmp_dict)
        # tmp_dict = df_train.groupby('due_date')['N_category'].mean()
        # test['진단명_target'] = test['due_date'].map(tmp_dict)
        # use_features += ['진단명_target']

        test = test[use_features].reset_index(drop=True)

        # lgb = LGBMClassifier(
        #                     n_estimators=5000,
        #                     learning_rate=0.03,
        #                     max_depth=-1,
        #                     num_leaves=64,
        #                     )

        # lgb.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=100)
        # oof[splits[fold][1]] += lgb.predict_proba(vv)[:, 1]
        # preds += lgb.predict_proba(test)[:, 1] / 5
        # fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, lgb.feature_importances_), columns=['feature', 'value'])])
        # tt[cat_features] = tt[cat_features].fillna(-99).astype(int)
        # vv[cat_features] = vv[cat_features].fillna(-99).astype(int)
        cat = CatBoostClassifier(
                                n_estimators=5000,
                                learning_rate=0.03,
                                # objective='logloss',
                                # eval_metric='F1',
                                reg_lambda=0.01,
                                random_seed=seed,
                                allow_writing_files=False,
                                logging_level='Silent',
                                )
        cat.fit(tt, tt_target, eval_set=[(tt, tt_target), (vv, vv_target)], verbose=1000, early_stopping_rounds=500,
                # cat_features=cat_features
                
                )
        oof[splits[fold][1]] += cat.predict_proba(vv)[:, 1]
        preds += cat.predict_proba(test)[:, 1] / FOLDS
        fi_df = pd.concat([fi_df, pd.DataFrame(zip(use_features, cat.feature_importances_), columns=['feature', 'value'])])
        
        
    seed_oof.append(oof)
    seed_pred.append(preds)

c:\Users\hhl\anaconda3\envs\ray_pl\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


c:\Users\hhl\anaconda3\envs\ray_pl\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


c:\Users\hhl\anaconda3\envs\ray_pl\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


c:\Users\hhl\anaconda3\envs\ray_pl\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


c:\Users\hhl\anaconda3\envs\ray_pl\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


start fold : 0
start fold : 1
start fold : 2
start fold : 3
start fold : 4


In [17]:
f1_score(df_train['N_category'],
         np.mean(seed_oof, 0).round(),
         average='macro')

0.8379837983798379

In [18]:
sub['N_category'] = np.where(np.mean(seed_pred, 0)>=0.5, 1, 0)

In [19]:
sub.to_csv('../submit/catboost__due_date__f1_basis.csv', index=False)